In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from dask.distributed import Client
import dask
import xarray as xr
import xgcm
from dask.diagnostics import ProgressBar
import sys
import matplotlib.patches as patches
sys.path.append('/homes/metofac/wenegrat/xroms/')
sys.path.append('/homes/metofac/wenegrat/pyspec/')

#from xroms import open_roms_netcdf_dataset
#import cmocean.cm as cmo
import xroms
#from pyspec import spectrum as spec
#from scipy import integrate as integrate
from timeit import default_timer as timer

#%%
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['font.size'] = 14
plt.rcParams['contour.negative_linestyle'] = 'solid'

import sys
sys.path.append("../src/")
import load_model_runs as lruns

from importlib import reload  
import time

/homes/metofac/wenegrat/xroms/xroms/interp.py:18: UserWarning: xESMF is not installed, so `interpll` will not run.
  warnings.warn("xESMF is not installed, so `interpll` will not run.")


In [2]:
client = Client(processes=False, n_workers=20, threads_per_worker=1, memory_limit='6GB')#(memory_limit='48GB')#Client(memory_limit='312GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://128.8.216.13:8787/status,
Dashboard: http://128.8.216.13:8787/status,Workers: 20
Total threads: 20,Total memory: 111.76 GiB
Status: running,Using processes: False
Comm: inproc://128.8.216.13/36894/1,Workers: 20
Dashboard: http://128.8.216.13:8787/status,Total threads: 20
Started: Just now,Total memory: 111.76 GiB
Comm: inproc://128.8.216.13/36894/7,Total threads: 1
Dashboard: http://128.8.216.13:32886/status,Memory: 5.59 GiB
Nanny: None,


In [3]:

def loadNewRun(filename, chunks, avg=False):
    # note here that filename should be the directory of history files
    from glob import glob

    
    if avg:
        files = sorted(glob(filename+'jet_avg.*.nc'))
    else:
        files = sorted(glob(filename+'jet_his.*.nc'))
    dsWC = xr.open_mfdataset(files, combine='nested', concat_dim='time', data_vars='minimal', chunks=chunks, coords='minimal', compat='override', parallel=True)

    #ds = xr.open_dataset('../data/model/jet_his.nc')

    dsWC, grid = xroms.roms_dataset(dsWC, Vtransform=1) ### IS THIS CORRECT???? NEED TO VERIFY!!
    print('and here')
    dsWC['rho'] = dsWC.temp
    dsWC = dsWC.swap_dims({'time':'ocean_time'})
    dsWC['temp'] = (-(1000+dsWC.rho)/(1000+dsWC.rho.isel(ocean_time=1, s_rho=-1).mean()) + 1)/2e-4 + 20
    return dsWC, grid

In [4]:
chunks={'time':1}

runname = 'JET_NML_CFB_BFLUX'
dsWC, gridWC = loadNewRun('../data/model/'+runname+'/', chunks=chunks, avg=False)
dsWC

runname = 'JET_NML_NOCFB_BFLUX'
dsNC, gridNC = loadNewRun('../data/model/'+runname+'/', chunks=chunks, avg=False)
dsNC

dsWC['buoyancy'] = -dsWC.rho
dsNC['buoyancy'] = -dsNC.rho

and here
and here


In [ ]:
dsWC['buoyancy'] = -dsWC.rho
dsNC['buoyancy'] = -dsNC.rho
zeta = dsWC.isel(ocean_time=-1, s_rho=slice(-4,-1), s_w=slice(-5,-1)).xroms.ertel
zeta2 = dsWC.isel(ocean_time=-1, s_rho=slice(-6,-2), s_w=slice(-7,-2)).xroms.ertel
print(zeta)
np.testing.assert_array_equal(zeta.isel(s_rho=0).compute(), zeta2.isel(s_rho=-1).compute())
zeta.isel(s_rho=0).plot(vmin = 0, vmax = 1e-7)

In [ ]:
ert = np.zeros((100, 4002, 1002))
for i in range(99, 90, -1):
    print(i)
    ert[i, :,:] = dsWC.isel(ocean_time=-1, s_rho=slice(i-3, i), s_w=slice(i-4,i)).xroms.ertel.compute().isel(s_rho=-1)

In [ ]:
ll = -50
ts = np.nan*np.zeros((42,))
for i in range(0, 42, 5):
    print(i)
    epvWC = dsWC.isel(ocean_time=i, s_rho=slice(ll, -1), s_w=slice(ll,-1)).xroms.ertel.sum().compute()
    epvNC = dsNC.isel(ocean_time=i, s_rho=slice(ll, -1), s_w=slice(ll,-1)).xroms.ertel.sum().compute()
    ts[i] =  (epvWC - epvNC)

0


2022-07-21 16:30:22,402 - distributed.worker_memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 4.51 GiB -- Worker memory limit: 5.59 GiB
2022-07-21 16:30:22,419 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.23 GiB -- Worker memory limit: 5.59 GiB
2022-07-21 16:30:22,500 - distributed.worker_memory - WARNING - Worker is at 93% memory usage. Pausing worker.  Process memory: 5.23 GiB -- Worker memory limit: 5.59 GiB
2022-07-21 16:30:22,502 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memo

In [ ]:
plt.plot(ts, marker='x')

In [ ]:
def select_interior(ds):
    """
    discard "exterior" u,v,rho-points to build a symetric grid
    
        Parameters:
            ds (xarray.Dataset): ROMS dataset
    """
    ds = ds.isel(xi_rho=slice(1,-1), eta_rho=slice(1,-1))
    if 'xi_v' in ds.dims:
        ds = ds.isel(xi_v=slice(1,-1))
    if 'eta_u' in ds.dims:
        ds = ds.isel(eta_u=slice(1,-1))
    if 'xi_u' in ds.dims:
        ds = ds.isel(xi_u=slice(0,-1))
    if 'eta_v' in ds.dims:
        ds = ds.isel(eta_v=slice(0,-1))
    return ds

dsWCn = select_interior(dsWC)
dsWCn

In [ ]:
coords = {
        "X": {"center": "xi_rho", "outer": "xi_u"},
        "Y": {"center": "eta_rho", "outer": "eta_v"},
        "Z": {"inner": "s_rho", "right": "s_w"},
    }

coords = {
        "X": {"center": "xi_rho", "left": "xi_u"},
        "Y": {"center": "eta_rho", "left": "eta_v"},
        "Z": {"inner": "s_rho", "right": "s_w"},
    }
grid2 = xgcm.Grid(dsWCn,coords=coords, periodic=['X'])
grid2

In [ ]:
# ABERNATHEY EXAMPLE
# https://github.com/xgcm/xgcm/issues/501


@xgcm.as_grid_ufunc(
     "(Y:center,X:left),(Y:center,X:left),(Y:left,X:center),(Y:left,X:center),(Y:left,X:left)->(Y:left,X:left)",
    #"(Y:center,X:outer),(Y:center,X:outer),(Y:outer,X:center),(Y:outer,X:center),(Y:outer,X:outer)->(Y:outer,X:outer)",
     boundary_width={"X": (1,0), "Y": (1,0)},
)
def vorticity(u, dxc, v, dyc, raz):
    u_trans = u * dxc
    v_trans = v * dyc
    v_diff_x = v_trans[..., 1:, 1:] - v_trans[..., 1:, :-1]
    u_diff_y = u_trans[..., 1:, 1:] - u_trans[..., :-1, 1:]
    #print((v_diff_x ).shape)
    #print((u_diff_y ).shape)
    #return v_diff_x, u_diff_y
    return (v_diff_x - u_diff_y)/raz[...,1:,1:]

vdiff = vorticity(grid2, dsWCn.u.isel(ocean_time=-1), dsWCn.dx_u, dsWCn.v.isel(ocean_time=-1), dsWCn.dy_v, dsWCn.dA_psi,  axis=5*[("Y", "X")], dask='parallelized')
vdiff.compute()
#vdiff.isel(s_rho=-1).plot(vmin=-1e-4, vmax=1e-4, cmap='RdBu_r')

In [ ]:
dsWCn

In [ ]:
dsWC
